<a href="https://colab.research.google.com/github/SwastikMethi/ResNet-50-vs.-VGG-16/blob/main/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tarfile
import urllib.request

# Download the Stanford Dogs dataset
url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'
dataset_dir = '/content/stanford-dogs'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    urllib.request.urlretrieve(url, os.path.join(dataset_dir, 'images.tar'))

# Extract the dataset
with tarfile.open(os.path.join(dataset_dir, 'images.tar'), 'r') as tar_ref:
    tar_ref.extractall(dataset_dir)

# --- The changes start here ---
import shutil
import random

# Create train and val directories
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split the data into train and validation sets (80% train, 20% val)
for breed_dir in os.listdir(os.path.join(dataset_dir, 'Images')):
    breed_path = os.path.join(dataset_dir, 'Images', breed_dir)
    images = os.listdir(breed_path)
    random.shuffle(images)
    split_index = int(len(images) * 0.8)

    # Move images to train directory
    for image in images[:split_index]:
        src = os.path.join(breed_path, image)
        dst = os.path.join(train_dir, breed_dir, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

    # Move images to val directory
    for image in images[split_index:]:
        src = os.path.join(breed_path, image)
        dst = os.path.join(val_dir, breed_dir, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

# Update data_dir to point to the new structure
data_dir = dataset_dir

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Path to dataset
data_dir = '/content/stanford-dogs'
batch_size = 16

# Load the dataset
image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x),
        transform=data_transforms[x]
    )
    for x in ['train', 'val']
}

# Create data loaders
dataloaders = {
    x: DataLoader(
        image_datasets[x],
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )
    for x in ['train', 'val']
}

# Load the pretrained VGG-16 model
model = models.vgg16(pretrained=True)

# Modify the last layer for 120 classes (Stanford Dogs dataset has 120 breeds)
num_classes = 120
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

# Move the model to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training parameters
num_epochs = 5
best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Training phase
    model.train()
    running_loss, running_corrects = 0.0, 0
    model.train()
    running_loss, running_corrects = 0.0, 0
    for batch_idx, (inputs, labels) in enumerate(dataloaders['train'], 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += (outputs.argmax(1) == labels).sum().item()

        # Print batch number
        print(f"  Batch {batch_idx}/{len(dataloaders['train'])} - Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects / len(image_datasets['train'])
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Validation phase
    model.eval()
    val_loss, val_corrects = 0.0, 0
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(image_datasets['val'])
    val_acc = val_corrects / len(image_datasets['val'])
    print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # Save the best model based on validation accuracy
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_vgg16_stanford_dogs.pth')

print("Training complete. Best validation accuracy: {:.4f}".format(best_acc))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also u

Streaming output truncated to the last 5000 lines.
  Batch 151/1027 - Loss: 2.6016
  Batch 152/1027 - Loss: 2.1034
  Batch 153/1027 - Loss: 2.8877
  Batch 154/1027 - Loss: 1.9650
  Batch 155/1027 - Loss: 2.7024
  Batch 156/1027 - Loss: 2.1558
  Batch 157/1027 - Loss: 1.8901
  Batch 158/1027 - Loss: 2.1669
  Batch 159/1027 - Loss: 1.8596
  Batch 160/1027 - Loss: 2.2334
  Batch 161/1027 - Loss: 1.8347
  Batch 162/1027 - Loss: 1.8385
  Batch 163/1027 - Loss: 2.1339
  Batch 164/1027 - Loss: 2.6387
  Batch 165/1027 - Loss: 2.3522
  Batch 166/1027 - Loss: 2.1252
  Batch 167/1027 - Loss: 2.7686
  Batch 168/1027 - Loss: 2.7328
  Batch 169/1027 - Loss: 3.2003
  Batch 170/1027 - Loss: 2.6766
  Batch 171/1027 - Loss: 2.2838
  Batch 172/1027 - Loss: 1.8916
  Batch 173/1027 - Loss: 2.2142
  Batch 174/1027 - Loss: 2.3243
  Batch 175/1027 - Loss: 2.6938
  Batch 176/1027 - Loss: 2.1664
  Batch 177/1027 - Loss: 2.4336
  Batch 178/1027 - Loss: 2.0380
  Batch 179/1027 - Loss: 2.5970
  Batch 180/1027 - Lo